## Encode flux and decode flux
The idea is to encode flux to a low dimension of about 4, use the econder to create extra input to a neural net by combining flux data and predictions of the encoder
runs with 3.8.2 on linux and 3.8.8 on mac

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Build data loader

In [2]:
import pyarrow.feather as feather
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd



In [3]:
def build_dataset(batchSizeTrain, batchsizeValid):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    # normalize input data
    fluxData_df_norm = (fluxData_df - fluxData_df.mean(axis=0)) / fluxData_df.std(axis=0)
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    X_train, X_val, y_train, y_val = train_test_split(fluxData_df_norm, fluxData_df_norm, test_size=0.2, random_state=42)


    train_target = torch.tensor(y_train.values.astype(np.float32))
    trainInput = torch.tensor(X_train.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(trainInput, train_target) 
    loaderTrain = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchSizeTrain, shuffle = True)


    valid_target = torch.tensor(y_val.values.astype(np.float32))
    validInput = torch.tensor(X_val.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(validInput, valid_target) 
    loaderValid = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchsizeValid, shuffle = True)
    return loaderTrain, loaderValid



# Build data loader

## Build AO network as class

In [4]:

class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder1 = nn.Linear(19,config.endcLayer1)
        self.encoder2 = nn.Linear(config.endcLayer1,config.endcLayer2)
        self.encoderOut = nn.Linear(config.endcLayer2, config.endcLayer3)



    def forward(self, x, act):
        x = self.encoder1(x)
        x = nn.ReLU(x)
        x = self.encoder2(x)
        x = nn.ReLU(x)
        x = self.encoderOut(x)
        return(x)


In [5]:

class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.decoder1 = nn.Linear(config.endcLayer3,config.decodLayer1)
        self.decoder1 = nn.Linear(config.decodLayer1,config.decodLayer2)
        self.decoderOut = nn.Linear(config.decodLayer2,config.endcLayer3)


    def forward(self, x, act):
        x = self.decoder1(x)
        x = nn.ReLU(x)
        x = self.decoder2(x)
        x = nn.ReLU(x)
        x = self.decoderOut(x)
        return(x)


In [6]:

class Autoencoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(19,config.endcLayer1),
            nn.Linear(config.endcLayer1,config.endcLayer2),
            nn.Linear(config.endcLayer2, config.endcLayer3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(config.endcLayer3,config.decodLayer1),
            nn.Linear(config.decodLayer1,config.decodLayer2),
            nn.Linear(config.decodLayer2,config.out)

        )


    def forward(self, x,):
        x = self.encoder(x)
        x = self.decoder(x)
        return(x)

In [7]:
import wandb

In [8]:
config = dict(
    epochs=5,
    batch_size=128,
    learning_rate=0.001,
    lrFactor=0.5,
    endcLayer1=100,
    endcLayer2=50,    
    endcLayer3=4,    
    decodLayer1=50,    
    decodLayer2=100,    
    out=19
  


    )

In [9]:
def train(config=None):
    print("next is init cmd")

    with wandb.init(project="EncodeFluxDecodeFlux", config=config):
        print("las was init cmd")
       # print(Encoder())

        config = wandb.config
        model = Autoencoder(config).to(device) 

        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=config.lrFactor, cooldown=5, patience=5,
        verbose=True)        
        print("next ist watch command")
        wandb.watch(model, log_freq=10)
        print("watch cmd finished")
        loaderTrain, loaderValid =build_dataset(1024,1024)
        for epoch in range(config.epochs):
            model.train()
            for i, data in enumerate(loaderTrain,0):
                input, labels = data
                input = input.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                loss = nn.MSELoss()
                model(input)
                loss =loss(model(input), labels)
        # ⬅ Backward pass + weight update
                loss.backward()
                optimizer.step()
            for i, data in enumerate(loaderValid,0):
                input, labels = data
                input = input.to(device)
                labels = labels.to(device)            
                model.eval()
                with torch.no_grad():
                    lossVal = nn.MSELoss()            
                    lossVal =lossVal(model(input), labels)
                    RmsLossValid=torch.sqrt(lossVal) 
            
            scheduler.step(RmsLossValid.item())  
            actualLR = optimizer.param_groups[0]["lr"]          
            print("epoch: ", epoch, "loss: ", "%.6f" % loss.item() ,"RmsLossValid: " , "%.6f" % RmsLossValid.item(), "Learning rate:", "%.8f" % actualLR)  

            wandb.log({"loss": loss, "RmsLossValid": RmsLossValid, "LearningRate": actualLR})

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            "RmsLossValid": RmsLossValid
            }, "PaperModel.pt")


In [10]:
train(config)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


next is init cmd


wandb: Currently logged in as: uwe-sterr (use `wandb login --relogin` to force relogin)


las was init cmd
next ist watch command
watch cmd finished
epoch:  0 loss:  0.372663 RmsLossValid:  0.612577 Learning rate: 0.00100000
epoch:  1 loss:  0.325009 RmsLossValid:  0.560718 Learning rate: 0.00100000
epoch:  2 loss:  0.324416 RmsLossValid:  0.554673 Learning rate: 0.00100000
epoch:  3 loss:  0.311529 RmsLossValid:  0.567765 Learning rate: 0.00100000
epoch:  4 loss:  0.321443 RmsLossValid:  0.566592 Learning rate: 0.00100000


loss,0.32144
RmsLossValid,0.56659
LearningRate,0.001
_runtime,10
_timestamp,1627292347
_step,4


loss,█▃▂▁▂
RmsLossValid,█▂▁▃▂
LearningRate,▁▁▁▁▁
_runtime,▁▁▅▅█
_timestamp,▁▁▅▅█
_step,▁▃▅▆█
